<a href="https://colab.research.google.com/github/KenObata/TISMIR_notebooks/blob/main/week15_RoBERTa_baseline_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook uses Pre-Trained RoBERTa model

Situation: English only (=multi-class).

Split: StratifiedKfold.



### set up

In [ ]:
!pip3 install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 6.8 MB/s 


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from collections import Counter

from skmultilearn.model_selection import IterativeStratification   
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score

def get_balanced_accuracy(model, McNemar, is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning):
  test_y = test.map(map_func_only_y)
  y_category=np.zeros((TEST_SIZE, ))
  counter=0
  for label_tensor in test_y.take(len(test_y)):
    y_test = np.argmax(label_tensor, axis=1)
    for label in y_test:
      y_category[counter]=label
      counter+=1

  X_test, y_test = test.map(map_func_only_X), y_category
  y_predict_test = np.asarray(model.predict(X_test))
  y_predict_test = np.argmax(y_predict_test, axis=1)
  print(classification_report(y_test, y_predict_test) )
  print(balanced_accuracy_score(y_test, y_predict_test))

  McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)] = []
  for ground_truh, pred in zip(y_test, y_predict_test):
        if ground_truh==pred:
          McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)].append(True)
        else:
          McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)].append(False)
  with open(DIR+ "RoBERTa_base_log.txt", "a") as f:
    print("======================================", file=f)
    print("is_fine_tuning?:", is_fine_tuning, "drop_out_rate: ", drop_out_rate, "learning_rate_transfer_learning: ", learning_rate_transfer_learning,
          "learning_rate_fine_tuning: ", learning_rate_fine_tuning, file=f)
    print(classification_report(y_test, y_predict_test) , file=f)
    print(balanced_accuracy_score(y_test, y_predict_test), file=f)

  return balanced_accuracy_score(y_test, y_predict_test), McNemar

In [ ]:
#save output into text
DIR = '/content/drive/MyDrive/music4all/'
with open(DIR+ "RoBERTa_base_log.txt", "a") as f:
  print("File name: week15_RoBERTa_baseline_keras.ipynb", file=f)


### Data Preparation(Kfold split)

Create dataframe for Gensim

In [ ]:
import pandas as pd

DIR = '/content/drive/MyDrive/music4all/'
df_genre_by_lang = pd.read_csv(DIR + 'df_genre_by_lang_full.csv')
df_genre_by_lang

,Unnamed: 0,id,genres,lang,lyric,number_of_line
0,0,0009fFIM1eYThaPg,pop,en,a sunny day so I got nowhere to hide Not a clo...,91
1,1,00P2bHdWFkghmDqz,soul,en,Tell me a tale that always was Sing me a song ...,36
2,2,00b6fV3nx5z2b8Ls,pop,en,A buh A buh You went to school to learn girl T...,74
3,3,013QDoTqbexEwkHr,pop,en,like a conversation where stops to breathe Is ...,20
4,4,01EKNot8qVgZpKM7,rock,en,Say the words I cannot say Say them on another...,31
...,...,...,...,...,...,...
13535,13535,zzT504Z94j1IAuc3,indie rock,en,think what afraid of come in you know been mad...,18
13536,13536,zzgS4ZqyswamEWNj,pop,en,Oh yeah yeah Last night I took a walk in the s...,75
13537,13537,zzx8CWdM7qkxKQpC,indie rock,en,Innocence it come easy in a sense it never wil...,34
13538,13538,zzz0n04uuTUA7fNh,pop,en,Girl you know how I feel I really Since you be...,65


In [ ]:
def load_data(df_col, y):
    texts, labels = [], []
    
    for line in df_col:
        # texts are already tokenized, just split on space
        # in a real use-case we would put more effort in preprocessing
        texts.append(line.split(' '))
    return pd.DataFrame({'texts': texts, 'labels': y})

In [ ]:
data = load_data(df_genre_by_lang["lyric"], df_genre_by_lang["genres"])

In [ ]:
data

,texts,labels
0,"[a, sunny, day, so, I, got, nowhere, to, hide,...",pop
1,"[Tell, me, a, tale, that, always, was, Sing, m...",soul
2,"[A, buh, A, buh, You, went, to, school, to, le...",pop
3,"[like, a, conversation, where, stops, to, brea...",pop
4,"[Say, the, words, I, cannot, say, Say, them, o...",rock
...,...,...
13535,"[think, what, afraid, of, come, in, you, know,...",indie rock
13536,"[Oh, yeah, yeah, Last, night, I, took, a, walk...",pop
13537,"[Innocence, it, come, easy, in, a, sense, it, ...",indie rock
13538,"[Girl, you, know, how, I, feel, I, really, Sin...",pop


In [ ]:
data['labels'] = data['labels'].astype('category')
label_mapping = data['labels'].cat.categories
data['labels'] = data['labels'].cat.codes
X = data['texts']
y = data['labels']

In [ ]:
type(X)

pandas.core.series.Series

In [ ]:
def StratifiedKFold_feature_and_df_glove(df, feature_list, y_name):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  y = df[y_name]
  skf.get_n_splits(df[ feature_list ], y)

  splits = []

  for train_index, test_index in skf.split(df[ feature_list ], y):
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = df[ feature_list ].loc[train_index], df[ feature_list ].loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [ ]:
def StratifiedKFold_feature_and_df(X, y):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  #y = df[y_name]
  skf.get_n_splits(X, y)#df[ feature_list ]

  splits = []

  for train_index, test_index in skf.split(X, y):#df[ feature_list ]
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = X.loc[train_index], X.loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [ ]:
#feature_list = ["texts"] #this is BOW and TF-IDF
#splits = StratifiedKFold_feature_and_df( data, feature_list, 'labels')
splits = StratifiedKFold_feature_and_df( X,y)

TRAIN: [    0     1     3 ... 13537 13538 13539] TEST: [    2     4     5 ... 13526 13532 13535]
TRAIN: [    0     2     4 ... 13535 13536 13539] TEST: [    1     3     7 ... 13530 13537 13538]
TRAIN: [    0     1     2 ... 13537 13538 13539] TEST: [    8    14    22 ... 13521 13531 13536]
TRAIN: [    0     1     2 ... 13537 13538 13539] TEST: [   10    12    15 ... 13523 13525 13534]
TRAIN: [    1     2     3 ... 13536 13537 13538] TEST: [    0     6    11 ... 13529 13533 13539]


In [ ]:
len(splits)

5

In [ ]:
split0=splits[0]
print(split0['X_train'].shape)
print(split0['y_train'].shape)
print(split0['X_test'].shape)
print(split0['y_test'].shape)

(10832,)
(10832,)
(2708,)
(2708,)


In [ ]:
split0['X_train']

0        [a, sunny, day, so, I, got, nowhere, to, hide,...
1        [Tell, me, a, tale, that, always, was, Sing, m...
3        [like, a, conversation, where, stops, to, brea...
6        [Locked, up, tight, Like, I, would, never, fee...
7        [sittin, in, the, crib, dreamin, about, leer, ...
                               ...                        
13534    [grandma, cookies, nigga, Shout, out, to, fron...
13536    [Oh, yeah, yeah, Last, night, I, took, a, walk...
13537    [Innocence, it, come, easy, in, a, sense, it, ...
13538    [Girl, you, know, how, I, feel, I, really, Sin...
13539    [wwI, oh, must, go, on, standing, You, break, ...
Name: texts, Length: 10832, dtype: object

In [ ]:
split0['y_train']

0        4
1        9
3        4
6        4
7        6
        ..
13534    6
13536    4
13537    3
13538    4
13539    8
Name: labels, Length: 10832, dtype: int8

### Use my self programmed balanced accuracy

In [ ]:
metrics = Metrics()
history = model.fit(train, validation_data=val, epochs=10, class_weight=my_weight ,callbacks=[metrics])
metrics.get_data()

Epoch 1/10
339/339 [==============================] - 225s 645ms/step - loss: 0.5565 - categorical_accuracy: 0.3878 - val_loss: 1.9337 - val_categorical_accuracy: 0.3273
Epoch 2/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5397 - categorical_accuracy: 0.3626 - val_loss: 2.0651 - val_categorical_accuracy: 0.2764
Epoch 3/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5375 - categorical_accuracy: 0.3392 - val_loss: 2.0931 - val_categorical_accuracy: 0.2273
Epoch 4/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5378 - categorical_accuracy: 0.3197 - val_loss: 2.0576 - val_categorical_accuracy: 0.2459
Epoch 5/10
339/339 [==============================] - 219s 645ms/step - loss: 0.5333 - categorical_accuracy: 0.3077 - val_loss: 2.0292 - val_categorical_accuracy: 0.2816
Epoch 6/10
339/339 [==============================] - 219s 646ms/step - loss: 0.5373 - categorical_accuracy: 0.2967 - val_loss: 2.1738 - val_categoric

[{'balanced_accuracy': 0.09744849916749872},
 {'balanced_accuracy': 0.10420636932765996},
 {'balanced_accuracy': 0.10970405736872939},
 {'balanced_accuracy': 0.10356561320774078},
 {'balanced_accuracy': 0.10792864804613309},
 {'balanced_accuracy': 0.09168989857066105},
 {'balanced_accuracy': 0.09926135499834263},
 {'balanced_accuracy': 0.10014630125778516},
 {'balanced_accuracy': 0.09807505457274496},
 {'balanced_accuracy': 0.09881853716030842}]

## From here, separate X_train, X_test from KFOldSplit

### Preprocess my lyrics data (Official train and test)

In [ ]:
import tensorflow as tf
!pip3 install transformers
SEQ_LEN = 256#512

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 15.8 MB/s 
     |████████████████████████████████| 6.6 MB 76.0 MB/s 
     |████████████████████████████████| 120 kB 51.9 MB/s 


In [ ]:
split0['X_test']

2        [A, buh, A, buh, You, went, to, school, to, le...
4        [Say, the, words, I, cannot, say, Say, them, o...
5        [I, was, alone, I, was, made, of, stone, You, ...
9        [Again, the, burden, of, losing, rests, upon, ...
20       [only, been, three, weeks, And, a, bag, of, sp...
                               ...                        
13517    [Like, the, legend, of, the, Phoenix, All, end...
13522    [Mr, Telephone, man, something, wrong, with, m...
13526    [can, you, imagine, what, it, would, be, like,...
13532    [Love, of, my, life, hurt, me, broken, my, hea...
13535    [think, what, afraid, of, come, in, you, know,...
Name: texts, Length: 2708, dtype: object

In [ ]:
def prepare_lyrics(X_series):
  for i, token_list in X_series.items():
    if type(token_list) is list:
      X_series.loc[i] = ' '.join(token_list)
  return X_series

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
SEQ_LEN=256

def get_Xid_Xmask(X_origin):
  Xids_train = np.zeros((X_origin.shape[0], SEQ_LEN))
  Xmask_train = np.zeros((X_origin.shape[0], SEQ_LEN))

  for i, lyric in enumerate(X_origin):
    tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
    Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']
  return Xids_train, Xmask_train

In [ ]:
def map_func(input_ids, masks, labels):
  return {'input_ids': input_ids, 'attention_mask':masks}, labels

In [ ]:
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
#tokenizer = AlbertTokenizer.from_pretrained('albert-large-v2')
Xids_train, Xmask_train = get_Xid_Xmask(split0['X_train'])
Xids_test, Xmask_test = get_Xid_Xmask(split0['X_test'])

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))
dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)

dataset_train = dataset_train.shuffle(42).batch(16)
DS_LEN = len(list(dataset_train))
SPLIT=0.9
train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

In [ ]:
counter = Counter(split0['y_train'])
SUM=0
for item in list(counter.values()) :
  SUM+=item
#SUM = sum(counter.values())
SUM

10832

In [ ]:
#Tutorial
#weight_for_0 = (1 / neg) * (total / 2.0)
#weight_for_1 = (1 / pos) * (total / 2.0)

counter = Counter(split0['y_train'])
my_weight2 = {}
print(counter)

for genre in counter:
  #print(genre, counter[genre])
  my_weight2[genre] = (1/counter[genre]) * (SUM/10)
my_weight2

Counter({4: 4143, 7: 1159, 9: 1030, 3: 865, 6: 783, 0: 763, 1: 690, 8: 556, 2: 537, 5: 306})


{4: 0.2614530533429882,
 9: 1.051650485436893,
 6: 1.383397190293742,
 2: 2.0171322160148977,
 0: 1.4196592398427261,
 7: 0.9345987920621226,
 1: 1.569855072463768,
 5: 3.539869281045752,
 8: 1.948201438848921,
 3: 1.2522543352601156}

In [ ]:
TEST_SIZE = len(split0['X_test'])

In [ ]:
def map_func_only_X(val_dictionary, labels):
  return {'input_ids': val_dictionary['input_ids'], 'attention_mask':val_dictionary['attention_mask']}

In [ ]:
def map_func_only_y(val_dictionary, labels):
  return labels

### First Fold Do parameter tuning for dropout rate

I have intentionally separated cells for check point purpose, based on dropout rates

In [ ]:
del(model1)

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.1
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    roberta = TFAutoModel.from_pretrained('roberta-base')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = roberta(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    print("Fine tuning---------------")
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc=get_balanced_accuracy(model1)
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc=get_balanced_accuracy(model1)
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.1


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Fine tuning---------------
Epoch 1/6
609/609 [==============================] - 158s 234ms/step - loss: 2.1254 - categorical_accuracy: 0.2421 - val_loss: 2.7890 - val_categorical_accuracy: 0.0588
Epoch 2/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9449 - categorical_accuracy: 0.2791 - val_loss: 2.3369 - val_categorical_accuracy: 0.1287
Epoch 3/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9025 - categorical_accuracy: 0.3010 - val_loss: 2.2305 - val_categorical_accuracy: 0.1342
Epoch 4/6
609/609 [==============================] - 138s 227ms/step - loss: 1.8750 - categorical_accuracy: 0.3058 - val_loss: 2.3698 - val_categorical_accuracy: 0.1572
Epoch 5/6
609/609 [==============================] - 138s 227ms/step - loss: 1.8419 - categorical_accuracy: 0.3216 - val_loss: 2.0977 - val_categorical_accuracy: 0.2188
Epoch 6/6
609/609 [==============================] - 138s 227ms/step - loss: 1.8247 - categorical_accuracy: 0.3221 - val_loss: 2

609/609 [==============================] - 355s 552ms/step - loss: 1.7651 - categorical_accuracy: 0.3520 - val_loss: 2.0853 - val_categorical_accuracy: 0.2822
Epoch 2/6
609/609 [==============================] - 333s 546ms/step - loss: 1.6665 - categorical_accuracy: 0.3747 - val_loss: 1.9222 - val_categorical_accuracy: 0.3318
Epoch 3/6
609/609 [==============================] - 333s 547ms/step - loss: 1.5449 - categorical_accuracy: 0.3892 - val_loss: 1.9141 - val_categorical_accuracy: 0.2960
Epoch 4/6
609/609 [==============================] - 333s 546ms/step - loss: 1.4398 - categorical_accuracy: 0.4226 - val_loss: 1.7410 - val_categorical_accuracy: 0.3888
Epoch 5/6
609/609 [==============================] - 333s 547ms/step - loss: 1.3479 - categorical_accuracy: 0.4450 - val_loss: 1.9958 - val_categorical_accuracy: 0.3281
Epoch 6/6
609/609 [==============================] - 333s 547ms/step - loss: 1.2593 - categorical_accuracy: 0.4690 - val_loss: 1.9778 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


...

609/609 [==============================] - 355s 552ms/step - loss: 1.8906 - categorical_accuracy: 0.2786 - val_loss: 2.6083 - val_categorical_accuracy: 0.1774
Epoch 2/6
609/609 [==============================] - 333s 546ms/step - loss: 1.8145 - categorical_accuracy: 0.3072 - val_loss: 2.2261 - val_categorical_accuracy: 0.2151
Epoch 3/6
609/609 [==============================] - 333s 546ms/step - loss: 1.7106 - categorical_accuracy: 0.3385 - val_loss: 2.1534 - val_categorical_accuracy: 0.2335
Epoch 4/6
609/609 [==============================] - 333s 546ms/step - loss: 1.6136 - categorical_accuracy: 0.3744 - val_loss: 2.1382 - val_categorical_accuracy: 0.2417
Epoch 5/6
609/609 [==============================] - 333s 546ms/step - loss: 1.5170 - categorical_accuracy: 0.3924 - val_loss: 1.9969 - val_categorical_accuracy: 0.2960
Epoch 6/6
609/609 [==============================] - 333s 546ms/step - loss: 1.4035 - categorical_accuracy: 0.4310 - val_loss: 2.1048 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Fine tuning---------------
Epoch 1/6
609/609 [==============================] - 159s 233ms/step - loss: 2.3831 - categorical_accuracy: 0.1111 - val_loss: 2.8378 - val_categorical_accuracy: 0.0708
Epoch 2/6
609/609 [==============================] - 138s 227ms/step - loss: 2.1767 - categorical_accuracy: 0.2061 - val_loss: 2.8430 - val_categorical_accuracy: 0.0790
Epoch 3/6
609/609 [==============================] - 138s 227ms/step - loss: 2.0884 - categorical_accuracy: 0.2331 - val_loss: 2.8152 - val_categorical_accuracy: 0.0754
Epoch 4/6
609/609 [==============================] - 138s 227ms/step - loss: 2.0327 - categorical_accuracy: 0.2445 - val_loss: 2.8686 - val_categorical_accuracy: 0.0873
Epoch 5/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9777 - categorical_accuracy: 0.2644 - val_loss: 2.7913 - val_categorical_accuracy: 0.1075
Epoch 6/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9413 - categorical_accuracy: 0.2704 - val_loss: 2

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


609/609 [==============================] - 356s 553ms/step - loss: 1.9194 - categorical_accuracy: 0.2842 - val_loss: 2.5078 - val_categorical_accuracy: 0.1526
Epoch 2/6
609/609 [==============================] - 334s 548ms/step - loss: 1.8499 - categorical_accuracy: 0.2932 - val_loss: 2.3879 - val_categorical_accuracy: 0.1728
Epoch 3/6
609/609 [==============================] - 333s 548ms/step - loss: 1.8278 - categorical_accuracy: 0.3077 - val_loss: 2.2629 - val_categorical_accuracy: 0.1967
Epoch 4/6
609/609 [==============================] - 333s 548ms/step - loss: 1.8011 - categorical_accuracy: 0.3098 - val_loss: 2.2230 - val_categorical_accuracy: 0.1967
Epoch 5/6
609/609 [==============================] - 334s 548ms/step - loss: 1.7662 - categorical_accuracy: 0.3263 - val_loss: 2.1091 - val_categorical_accuracy: 0.2344
Epoch 6/6
609/609 [==============================] - 334s 548ms/step - loss: 1.7279 - categorical_accuracy: 0.3383 - val_loss: 2.1070 - val_categorical_accuracy: 0.2

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.2
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    roberta = TFAutoModel.from_pretrained('roberta-base')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = roberta(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model2.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    print("Fine tuning---------------")
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc=get_balanced_accuracy(model2)
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    model2.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc=get_balanced_accuracy(model2)
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model2)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.2


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Fine tuning---------------
Epoch 1/6
609/609 [==============================] - 160s 234ms/step - loss: 2.1581 - categorical_accuracy: 0.2218 - val_loss: 2.5023 - val_categorical_accuracy: 0.0864
Epoch 2/6
609/609 [==============================] - 139s 228ms/step - loss: 1.9832 - categorical_accuracy: 0.2747 - val_loss: 2.4512 - val_categorical_accuracy: 0.0938
Epoch 3/6
609/609 [==============================] - 139s 228ms/step - loss: 1.9134 - categorical_accuracy: 0.2979 - val_loss: 2.4011 - val_categorical_accuracy: 0.1388
Epoch 4/6
609/609 [==============================] - 139s 228ms/step - loss: 1.8976 - categorical_accuracy: 0.2991 - val_loss: 2.2255 - val_categorical_accuracy: 0.1710
Epoch 5/6
609/609 [==============================] - 139s 228ms/step - loss: 1.8614 - categorical_accuracy: 0.3014 - val_loss: 2.0955 - val_categorical_accuracy: 0.2298
Epoch 6/6
609/609 [==============================] - 139s 228ms/step - loss: 1.8444 - categorical_accuracy: 0.3021 - val_loss: 2

609/609 [==============================] - 356s 553ms/step - loss: 1.7836 - categorical_accuracy: 0.3286 - val_loss: 1.9110 - val_categorical_accuracy: 0.2923
Epoch 2/6
609/609 [==============================] - 333s 547ms/step - loss: 1.6761 - categorical_accuracy: 0.3612 - val_loss: 1.7647 - val_categorical_accuracy: 0.3621
Epoch 3/6
609/609 [==============================] - 333s 547ms/step - loss: 1.5659 - categorical_accuracy: 0.3842 - val_loss: 1.9138 - val_categorical_accuracy: 0.3254
Epoch 4/6
609/609 [==============================] - 333s 547ms/step - loss: 1.4739 - categorical_accuracy: 0.4050 - val_loss: 1.9317 - val_categorical_accuracy: 0.3033
Epoch 5/6
609/609 [==============================] - 333s 547ms/step - loss: 1.3854 - categorical_accuracy: 0.4290 - val_loss: 1.8743 - val_categorical_accuracy: 0.3153
              precision    recall  f1-score   support

         0.0       0.22      0.27      0.24       190
         1.0       0.19      0.56      0.28       173
  

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


...

Fine tuning---------------
Epoch 1/6
609/609 [==============================] - 157s 233ms/step - loss: 2.4506 - categorical_accuracy: 0.1486 - val_loss: 2.4632 - val_categorical_accuracy: 0.0800
Epoch 2/6
609/609 [==============================] - 139s 227ms/step - loss: 2.2100 - categorical_accuracy: 0.2175 - val_loss: 2.6189 - val_categorical_accuracy: 0.0717
Epoch 3/6
609/609 [==============================] - 139s 228ms/step - loss: 2.1196 - categorical_accuracy: 0.2400 - val_loss: 2.6430 - val_categorical_accuracy: 0.0781
Epoch 4/6
609/609 [==============================] - 139s 228ms/step - loss: 2.0654 - categorical_accuracy: 0.2500 - val_loss: 2.7605 - val_categorical_accuracy: 0.0726
              precision    recall  f1-score   support

         0.0       0.09      0.25      0.13       190
         1.0       0.10      0.28      0.14       173
         2.0       0.06      0.03      0.04       135
         3.0       0.06      0.02      0.03       216
         4.0       0.50   

609/609 [==============================] - 358s 553ms/step - loss: 1.9479 - categorical_accuracy: 0.2932 - val_loss: 2.5098 - val_categorical_accuracy: 0.1526
Epoch 2/6
609/609 [==============================] - 333s 547ms/step - loss: 1.8566 - categorical_accuracy: 0.2903 - val_loss: 2.5891 - val_categorical_accuracy: 0.1774
Epoch 3/6
609/609 [==============================] - 333s 547ms/step - loss: 1.7687 - categorical_accuracy: 0.3219 - val_loss: 2.8941 - val_categorical_accuracy: 0.1535
Epoch 4/6
609/609 [==============================] - 333s 547ms/step - loss: 1.6688 - categorical_accuracy: 0.3422 - val_loss: 1.8996 - val_categorical_accuracy: 0.2932
Epoch 5/6
609/609 [==============================] - 333s 547ms/step - loss: 1.5842 - categorical_accuracy: 0.3717 - val_loss: 2.3144 - val_categorical_accuracy: 0.2316
Epoch 6/6
609/609 [==============================] - 333s 547ms/step - loss: 1.4828 - categorical_accuracy: 0.4020 - val_loss: 1.9403 - val_categorical_accuracy: 0.3

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Fine tuning---------------
Epoch 1/6
609/609 [==============================] - 157s 234ms/step - loss: 2.4019 - categorical_accuracy: 0.1797 - val_loss: 2.5544 - val_categorical_accuracy: 0.0643
Epoch 2/6
609/609 [==============================] - 139s 228ms/step - loss: 2.1694 - categorical_accuracy: 0.2142 - val_loss: 2.5657 - val_categorical_accuracy: 0.0790
Epoch 3/6
609/609 [==============================] - 139s 228ms/step - loss: 2.0991 - categorical_accuracy: 0.2307 - val_loss: 2.5950 - val_categorical_accuracy: 0.0864
Epoch 4/6
609/609 [==============================] - 139s 228ms/step - loss: 2.0585 - categorical_accuracy: 0.2467 - val_loss: 2.5448 - val_categorical_accuracy: 0.0882
Epoch 5/6
609/609 [==============================] - 139s 228ms/step - loss: 2.0018 - categorical_accuracy: 0.2630 - val_loss: 2.5103 - val_categorical_accuracy: 0.0938
Epoch 6/6
609/609 [==============================] - 139s 228ms/step - loss: 1.9966 - categorical_accuracy: 0.2611 - val_loss: 2

609/609 [==============================] - 356s 552ms/step - loss: 1.9419 - categorical_accuracy: 0.2669 - val_loss: 2.3577 - val_categorical_accuracy: 0.1425
Epoch 2/6
609/609 [==============================] - 333s 547ms/step - loss: 1.8903 - categorical_accuracy: 0.2873 - val_loss: 2.2204 - val_categorical_accuracy: 0.1765
Epoch 3/6
609/609 [==============================] - 333s 547ms/step - loss: 1.8500 - categorical_accuracy: 0.3022 - val_loss: 2.1369 - val_categorical_accuracy: 0.2371
Epoch 4/6
609/609 [==============================] - 333s 547ms/step - loss: 1.8275 - categorical_accuracy: 0.3140 - val_loss: 2.0907 - val_categorical_accuracy: 0.2426
Epoch 5/6
609/609 [==============================] - 333s 547ms/step - loss: 1.8033 - categorical_accuracy: 0.3242 - val_loss: 2.0726 - val_categorical_accuracy: 0.2555
Epoch 6/6
609/609 [==============================] - 333s 547ms/step - loss: 1.7753 - categorical_accuracy: 0.3251 - val_loss: 2.0157 - val_categorical_accuracy: 0.2

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.3
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    roberta = TFAutoModel.from_pretrained('roberta-base')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = roberta(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model2.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    
    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate, learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model2.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model2)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.3


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 157s 233ms/step - loss: 2.1858 - categorical_accuracy: 0.1974 - val_loss: 2.7666 - val_categorical_accuracy: 0.0414
Epoch 2/6
609/609 [==============================] - 138s 227ms/step - loss: 2.0308 - categorical_accuracy: 0.2525 - val_loss: 2.4305 - val_categorical_accuracy: 0.1149
Epoch 3/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9643 - categorical_accuracy: 0.2895 - val_loss: 2.3613 - val_categorical_accuracy: 0.1259
Epoch 4/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9265 - categorical_accuracy: 0.2989 - val_loss: 2.1383 - val_categorical_accuracy: 0.1792
Epoch 5/6
609/609 [==============================] - 138s 226ms/step - loss: 1.8929 - categorical_accuracy: 0.2921 - val_loss: 2.3129 - val_categorical_accuracy: 0.1590
Epoch 6/6
609/609 [==============================] - 138s 227ms/step - loss: 1.8842 - categorical_accuracy: 0.3060 - val_loss: 2.1537 - val_categorical_acc

609/609 [==============================] - 356s 553ms/step - loss: 1.8342 - categorical_accuracy: 0.3171 - val_loss: 1.8368 - val_categorical_accuracy: 0.3079
Epoch 2/6
609/609 [==============================] - 333s 547ms/step - loss: 1.7321 - categorical_accuracy: 0.3358 - val_loss: 1.8738 - val_categorical_accuracy: 0.3208
Epoch 3/6
609/609 [==============================] - 333s 547ms/step - loss: 1.6249 - categorical_accuracy: 0.3608 - val_loss: 1.7893 - val_categorical_accuracy: 0.3428
Epoch 4/6
609/609 [==============================] - 333s 547ms/step - loss: 1.5474 - categorical_accuracy: 0.3836 - val_loss: 1.8064 - val_categorical_accuracy: 0.3401
Epoch 5/6
609/609 [==============================] - 333s 547ms/step - loss: 1.4475 - categorical_accuracy: 0.3967 - val_loss: 1.7726 - val_categorical_accuracy: 0.3079
Epoch 6/6
609/609 [==============================] - 333s 547ms/step - loss: 1.3779 - categorical_accuracy: 0.4176 - val_loss: 1.7867 - val_categorical_accuracy: 0.3

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


...

Epoch 1/6
609/609 [==============================] - 158s 234ms/step - loss: 2.4675 - categorical_accuracy: 0.1430 - val_loss: 2.5890 - val_categorical_accuracy: 0.1011
Epoch 2/6
609/609 [==============================] - 139s 228ms/step - loss: 2.2370 - categorical_accuracy: 0.1887 - val_loss: 2.6591 - val_categorical_accuracy: 0.0974
Epoch 3/6
609/609 [==============================] - 139s 228ms/step - loss: 2.1290 - categorical_accuracy: 0.2217 - val_loss: 2.6546 - val_categorical_accuracy: 0.0892
Epoch 4/6
609/609 [==============================] - 139s 228ms/step - loss: 2.0813 - categorical_accuracy: 0.2274 - val_loss: 2.6044 - val_categorical_accuracy: 0.0947
              precision    recall  f1-score   support

         0.0       0.08      0.41      0.14       190
         1.0       0.12      0.04      0.06       173
         2.0       0.00      0.00      0.00       135
         3.0       0.06      0.12      0.08       216
         4.0       0.00      0.00      0.00      1036

609/609 [==============================] - 356s 553ms/step - loss: 1.9973 - categorical_accuracy: 0.2602 - val_loss: 2.6412 - val_categorical_accuracy: 0.1618
Epoch 2/6
609/609 [==============================] - 334s 548ms/step - loss: 1.9062 - categorical_accuracy: 0.2854 - val_loss: 2.3787 - val_categorical_accuracy: 0.2013
Epoch 3/6
609/609 [==============================] - 334s 548ms/step - loss: 1.8095 - categorical_accuracy: 0.3136 - val_loss: 2.7480 - val_categorical_accuracy: 0.2031
Epoch 4/6
609/609 [==============================] - 334s 548ms/step - loss: 1.7497 - categorical_accuracy: 0.3179 - val_loss: 2.8453 - val_categorical_accuracy: 0.1903
Epoch 5/6
609/609 [==============================] - 334s 548ms/step - loss: 1.6493 - categorical_accuracy: 0.3539 - val_loss: 2.5189 - val_categorical_accuracy: 0.2298
Epoch 6/6
609/609 [==============================] - 334s 548ms/step - loss: 1.5639 - categorical_accuracy: 0.3779 - val_loss: 2.3980 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 157s 233ms/step - loss: 2.4211 - categorical_accuracy: 0.1604 - val_loss: 2.5151 - val_categorical_accuracy: 0.0634
Epoch 2/6
609/609 [==============================] - 138s 227ms/step - loss: 2.2252 - categorical_accuracy: 0.2009 - val_loss: 2.5621 - val_categorical_accuracy: 0.0708
Epoch 3/6
609/609 [==============================] - 138s 227ms/step - loss: 2.1465 - categorical_accuracy: 0.2208 - val_loss: 2.6552 - val_categorical_accuracy: 0.0763
Epoch 4/6
609/609 [==============================] - 138s 227ms/step - loss: 2.0943 - categorical_accuracy: 0.2401 - val_loss: 2.6740 - val_categorical_accuracy: 0.0699
Epoch 5/6
609/609 [==============================] - 138s 227ms/step - loss: 2.0486 - categorical_accuracy: 0.2527 - val_loss: 2.7883 - val_categorical_accuracy: 0.0588
Epoch 6/6
609/609 [==============================] - 138s 227ms/step - loss: 2.0409 - categorical_accuracy: 0.2487 - val_loss: 2.6652 - val_categorical_acc

609/609 [==============================] - 358s 553ms/step - loss: 1.9968 - categorical_accuracy: 0.2655 - val_loss: 2.4237 - val_categorical_accuracy: 0.1379
Epoch 2/6
609/609 [==============================] - 333s 547ms/step - loss: 1.9558 - categorical_accuracy: 0.2766 - val_loss: 2.3208 - val_categorical_accuracy: 0.1857
Epoch 3/6
609/609 [==============================] - 333s 547ms/step - loss: 1.9146 - categorical_accuracy: 0.2956 - val_loss: 2.1240 - val_categorical_accuracy: 0.2335
Epoch 4/6
609/609 [==============================] - 333s 547ms/step - loss: 1.8903 - categorical_accuracy: 0.3016 - val_loss: 2.0956 - val_categorical_accuracy: 0.2426
Epoch 5/6
609/609 [==============================] - 333s 547ms/step - loss: 1.8671 - categorical_accuracy: 0.3089 - val_loss: 2.1063 - val_categorical_accuracy: 0.2371
Epoch 6/6
609/609 [==============================] - 333s 547ms/step - loss: 1.8673 - categorical_accuracy: 0.3074 - val_loss: 2.1055 - val_categorical_accuracy: 0.2

In [ ]:
balanced_accuracies_fine_tuning

[0.3545180413326312,
 0.32561995928565546,
 0.3198687138125366,
 0.2922621302487614]

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
#balanced_accuracies_transfer_learning = []
#balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.4
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    roberta = TFAutoModel.from_pretrained('roberta-base')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = roberta(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model2.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    
    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate, learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model2.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model2)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.4


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 157s 232ms/step - loss: 2.2192 - categorical_accuracy: 0.2211 - val_loss: 2.6437 - val_categorical_accuracy: 0.0864
Epoch 2/6
609/609 [==============================] - 138s 227ms/step - loss: 2.0214 - categorical_accuracy: 0.2580 - val_loss: 2.3976 - val_categorical_accuracy: 0.0947
Epoch 3/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9585 - categorical_accuracy: 0.2678 - val_loss: 2.4780 - val_categorical_accuracy: 0.0974
Epoch 4/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9317 - categorical_accuracy: 0.2800 - val_loss: 2.2564 - val_categorical_accuracy: 0.1756
Epoch 5/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9000 - categorical_accuracy: 0.2917 - val_loss: 2.1081 - val_categorical_accuracy: 0.1976
Epoch 6/6
609/609 [==============================] - 138s 227ms/step - loss: 1.8928 - categorical_accuracy: 0.2923 - val_loss: 2.0816 - val_categorical_acc

609/609 [==============================] - 358s 553ms/step - loss: 1.8378 - categorical_accuracy: 0.2965 - val_loss: 2.1463 - val_categorical_accuracy: 0.2132
Epoch 2/6
609/609 [==============================] - 333s 548ms/step - loss: 1.7394 - categorical_accuracy: 0.3235 - val_loss: 1.8245 - val_categorical_accuracy: 0.3097
Epoch 3/6
609/609 [==============================] - 333s 547ms/step - loss: 1.6453 - categorical_accuracy: 0.3526 - val_loss: 1.8455 - val_categorical_accuracy: 0.2932
Epoch 4/6
609/609 [==============================] - 333s 547ms/step - loss: 1.5706 - categorical_accuracy: 0.3671 - val_loss: 1.9466 - val_categorical_accuracy: 0.2472
Epoch 5/6
609/609 [==============================] - 333s 547ms/step - loss: 1.4801 - categorical_accuracy: 0.3900 - val_loss: 1.9108 - val_categorical_accuracy: 0.2868
              precision    recall  f1-score   support

         0.0       0.24      0.24      0.24       190
         1.0       0.20      0.53      0.29       173
  

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

609/609 [==============================] - 358s 553ms/step - loss: 2.0634 - categorical_accuracy: 0.2356 - val_loss: 4.4193 - val_categorical_accuracy: 0.0267
Epoch 2/6
609/609 [==============================] - 333s 547ms/step - loss: 2.0384 - categorical_accuracy: 0.2349 - val_loss: 4.3172 - val_categorical_accuracy: 0.1305
Epoch 3/6
609/609 [==============================] - 333s 547ms/step - loss: 1.9487 - categorical_accuracy: 0.2601 - val_loss: 3.2020 - val_categorical_accuracy: 0.1461
Epoch 4/6
609/609 [==============================] - 334s 548ms/step - loss: 1.8867 - categorical_accuracy: 0.2733 - val_loss: 2.9971 - val_categorical_accuracy: 0.1498
Epoch 5/6
609/609 [==============================] - 333s 548ms/step - loss: 1.8058 - categorical_accuracy: 0.3124 - val_loss: 2.9632 - val_categorical_accuracy: 0.1608
Epoch 6/6
609/609 [==============================] - 334s 548ms/step - loss: 1.7531 - categorical_accuracy: 0.3319 - val_loss: 2.7324 - val_categorical_accuracy: 0.1

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/6
147/609 [======>.......................] - ETA: 1:35 - loss: 2.7419 - categorical_accuracy: 0.1054

KeyboardInterrupt: ignored

In [ ]:
balanced_accuracies_fine_tuning

[0.3545180413326312,
 0.32561995928565546,
 0.3198687138125366,
 0.2922621302487614,
 0.35156416694413817,
 0.3320729042652105,
 0.3001390737921044]

### Resume from the last cell, dropout rate=0.4

In [ ]:
drop_out_rate = 0.4
learning_rate_transfer_learning =1e-4
learning_rate_fine_tuning = 1e-6

In [ ]:
McNemar={}

In [ ]:
balanced_accuracies_transfer_learning=[]
balanced_accuracies_fine_tuning = []

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
#balanced_accuracies_transfer_learning = []
#balanced_accuracies_fine_tuning = []
SEQ_LEN2=256


print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
, learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
#step1
roberta = TFAutoModel.from_pretrained('roberta-base')
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = roberta(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model2.layers[2].trainable = False
#model2.summary()

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
#model2.summary() #Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate, learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_transfer_learning.append( balanced_acc )

#step5: fine tune
print("Fine tuning---------------")
model2.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_fine_tuning.append( balanced_acc )
print("----------------------------------------")
del(model2)


learning_rate_transfer_learning:  0.0001 learning_rate_fine_tuning:  1e-06 drop_out_rate: 0.4


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 97s 138ms/step - loss: 2.5303 - categorical_accuracy: 0.1476 - val_loss: 2.4646 - val_categorical_accuracy: 0.0901
Epoch 2/6
609/609 [==============================] - 81s 132ms/step - loss: 2.2815 - categorical_accuracy: 0.1943 - val_loss: 2.5647 - val_categorical_accuracy: 0.1075
Epoch 3/6
609/609 [==============================] - 81s 132ms/step - loss: 2.1948 - categorical_accuracy: 0.2070 - val_loss: 2.5388 - val_categorical_accuracy: 0.1039
Epoch 4/6
609/609 [==============================] - 80s 132ms/step - loss: 2.1348 - categorical_accuracy: 0.2227 - val_loss: 2.6419 - val_categorical_accuracy: 0.0864
Epoch 5/6
609/609 [==============================] - 81s 133ms/step - loss: 2.0975 - categorical_accuracy: 0.2318 - val_loss: 2.5691 - val_categorical_accuracy: 0.0901
              precision    recall  f1-score   support

         0.0       0.10      0.30      0.15       190
         1.0       0.08      0.19      0.11       1

609/609 [==============================] - 208s 311ms/step - loss: 2.0607 - categorical_accuracy: 0.2404 - val_loss: 2.4476 - val_categorical_accuracy: 0.1250
Epoch 2/6
609/609 [==============================] - 186s 306ms/step - loss: 2.0153 - categorical_accuracy: 0.2582 - val_loss: 2.3558 - val_categorical_accuracy: 0.1452
Epoch 3/6
609/609 [==============================] - 187s 306ms/step - loss: 1.9797 - categorical_accuracy: 0.2610 - val_loss: 2.2952 - val_categorical_accuracy: 0.1792
Epoch 4/6
609/609 [==============================] - 186s 306ms/step - loss: 1.9457 - categorical_accuracy: 0.2721 - val_loss: 2.1558 - val_categorical_accuracy: 0.2316
Epoch 5/6
609/609 [==============================] - 186s 306ms/step - loss: 1.9300 - categorical_accuracy: 0.2826 - val_loss: 2.0924 - val_categorical_accuracy: 0.2537
Epoch 6/6
609/609 [==============================] - 186s 306ms/step - loss: 1.9115 - categorical_accuracy: 0.2891 - val_loss: 2.1359 - val_categorical_accuracy: 0.2

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
#balanced_accuracies_transfer_learning = []
#balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.5
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    roberta = TFAutoModel.from_pretrained('roberta-base')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = roberta(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model2.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    
    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate, learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model2.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model2)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.5


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 97s 138ms/step - loss: 2.2807 - categorical_accuracy: 0.1991 - val_loss: 2.4103 - val_categorical_accuracy: 0.1057
Epoch 2/6
609/609 [==============================] - 81s 132ms/step - loss: 2.0433 - categorical_accuracy: 0.2418 - val_loss: 2.3670 - val_categorical_accuracy: 0.1085
Epoch 3/6
609/609 [==============================] - 81s 132ms/step - loss: 1.9955 - categorical_accuracy: 0.2618 - val_loss: 2.3110 - val_categorical_accuracy: 0.1498
Epoch 4/6
609/609 [==============================] - 81s 133ms/step - loss: 1.9470 - categorical_accuracy: 0.2659 - val_loss: 2.2653 - val_categorical_accuracy: 0.1526
Epoch 5/6
609/609 [==============================] - 81s 133ms/step - loss: 1.9342 - categorical_accuracy: 0.2788 - val_loss: 2.2505 - val_categorical_accuracy: 0.1930
Epoch 6/6
609/609 [==============================] - 81s 132ms/step - loss: 1.9028 - categorical_accuracy: 0.2952 - val_loss: 2.1968 - val_categorical_accuracy:

609/609 [==============================] - 209s 312ms/step - loss: 1.8468 - categorical_accuracy: 0.3036 - val_loss: 1.9654 - val_categorical_accuracy: 0.2656
Epoch 2/6
609/609 [==============================] - 187s 307ms/step - loss: 1.7603 - categorical_accuracy: 0.3356 - val_loss: 2.0343 - val_categorical_accuracy: 0.2518
Epoch 3/6
609/609 [==============================] - 187s 307ms/step - loss: 1.6907 - categorical_accuracy: 0.3449 - val_loss: 2.0835 - val_categorical_accuracy: 0.2482
Epoch 4/6
609/609 [==============================] - 187s 307ms/step - loss: 1.6170 - categorical_accuracy: 0.3677 - val_loss: 2.0746 - val_categorical_accuracy: 0.2500
              precision    recall  f1-score   support

         0.0       0.26      0.17      0.20       190
         1.0       0.15      0.77      0.25       173
         2.0       0.05      0.02      0.03       135
         3.0       0.22      0.19      0.20       216
         4.0       0.74      0.14      0.24      1036
         

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


...

609/609 [==============================] - 208s 313ms/step - loss: 2.0222 - categorical_accuracy: 0.2757 - val_loss: 2.6481 - val_categorical_accuracy: 0.1314
Epoch 2/6
609/609 [==============================] - 187s 307ms/step - loss: 1.9442 - categorical_accuracy: 0.2881 - val_loss: 2.6962 - val_categorical_accuracy: 0.1535
Epoch 3/6
609/609 [==============================] - 187s 307ms/step - loss: 1.9018 - categorical_accuracy: 0.3047 - val_loss: 3.5976 - val_categorical_accuracy: 0.1268
Epoch 4/6
609/609 [==============================] - 187s 307ms/step - loss: 1.8311 - categorical_accuracy: 0.3293 - val_loss: 3.3645 - val_categorical_accuracy: 0.1287
Epoch 5/6
609/609 [==============================] - 187s 307ms/step - loss: 1.7641 - categorical_accuracy: 0.3519 - val_loss: 3.0120 - val_categorical_accuracy: 0.1572
Epoch 6/6
609/609 [==============================] - 187s 307ms/step - loss: 1.7074 - categorical_accuracy: 0.3653 - val_loss: 3.1106 - val_categorical_accuracy: 0.1

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 98s 139ms/step - loss: 2.6928 - categorical_accuracy: 0.1470 - val_loss: 2.7250 - val_categorical_accuracy: 0.0781
Epoch 2/6
609/609 [==============================] - 81s 133ms/step - loss: 2.3846 - categorical_accuracy: 0.1722 - val_loss: 2.6633 - val_categorical_accuracy: 0.0910
Epoch 3/6
609/609 [==============================] - 81s 133ms/step - loss: 2.2742 - categorical_accuracy: 0.1835 - val_loss: 2.5558 - val_categorical_accuracy: 0.1066
Epoch 4/6
609/609 [==============================] - 81s 133ms/step - loss: 2.2139 - categorical_accuracy: 0.1926 - val_loss: 2.5263 - val_categorical_accuracy: 0.1121
Epoch 5/6
609/609 [==============================] - 81s 133ms/step - loss: 2.1583 - categorical_accuracy: 0.2108 - val_loss: 2.5198 - val_categorical_accuracy: 0.1112
Epoch 6/6
609/609 [==============================] - 81s 133ms/step - loss: 2.1558 - categorical_accuracy: 0.2101 - val_loss: 2.5200 - val_categorical_accuracy:

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

609/609 [==============================] - 207s 312ms/step - loss: 2.0931 - categorical_accuracy: 0.2244 - val_loss: 2.4564 - val_categorical_accuracy: 0.1434
Epoch 2/6
609/609 [==============================] - 187s 307ms/step - loss: 2.0462 - categorical_accuracy: 0.2373 - val_loss: 2.3063 - val_categorical_accuracy: 0.1691
Epoch 3/6
609/609 [==============================] - 187s 307ms/step - loss: 2.0093 - categorical_accuracy: 0.2498 - val_loss: 2.2745 - val_categorical_accuracy: 0.1756
Epoch 4/6
609/609 [==============================] - 187s 307ms/step - loss: 1.9851 - categorical_accuracy: 0.2623 - val_loss: 2.2025 - val_categorical_accuracy: 0.1958
Epoch 5/6
609/609 [==============================] - 186s 306ms/step - loss: 1.9666 - categorical_accuracy: 0.2636 - val_loss: 2.1953 - val_categorical_accuracy: 0.2050
Epoch 6/6
609/609 [==============================] - 186s 305ms/step - loss: 1.9550 - categorical_accuracy: 0.2722 - val_loss: 2.0923 - val_categorical_accuracy: 0.2

In [ ]:
print(balanced_accuracies_transfer_learning)
print(balanced_accuracies_fine_tuning )

[0.1545847274897451, 0.24177789377190567, 0.2388573750703392, 0.13906206969618845, 0.17849565301702414]
[0.27549872208426807, 0.32026688821404903, 0.3204359843828537, 0.28664294751526975, 0.29482717944210146]


### KFold test

Best parameter set is: initial learning rate=1e-3, fine tuning learning rate=1e-5, drop out rate=0.2 

In [ ]:
del(dataset_train)
del(dataset_test)
del(train)
del(val)
del(test)

#### Fold1: intentionally separate cells for check point purpose.

In [ ]:
from transformers import AutoTokenizer
import numpy as np

split0=splits[1]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
SEQ_LEN=256
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)
Xids_train.shape, Xids_test.shape:  (10832, 256) (2708, 256)


In [ ]:
drop_out_rate = 0.2
learning_rate_transfer_learning =1e-3
learning_rate_fine_tuning = 1e-5

In [ ]:
#if len(balanced_accuracy_transfer_learning)==0:
balanced_accuracy_transfer_learning = []
#if len(balanced_accuracy_fine_tuning)==0:
balanced_accuracy_fine_tune = []

In [ ]:
McNemar.keys()

dict_keys([(False, 0.4, 0.0001, 1e-06), (True, 0.4, 0.0001, 1e-06), (False, 0.5, 0.001, 1e-05), (True, 0.5, 0.001, 1e-05), (False, 0.5, 0.001, 1e-06), (True, 0.5, 0.001, 1e-06), (False, 0.5, 0.0001, 1e-05), (True, 0.5, 0.0001, 1e-05), (False, 0.5, 0.0001, 1e-06), (True, 0.5, 0.0001, 1e-06)])

In [ ]:
from transformers import TFAutoModel

roberta_base = TFAutoModel.from_pretrained('roberta-base')
SEQ_LEN2=256
#step1
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = roberta_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model2.layers[2].trainable = False
print(model2.summary())

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
print(model2.summary() )#Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_transfer_learning.append(balanced_acc)


#step5: fine tune
model2.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_fine_tune.append(balanced_acc)
del(model2)


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_roberta_model_7 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 odel)                          thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

609/609 [==============================] - 205s 310ms/step - loss: 1.7878 - categorical_accuracy: 0.3159 - val_loss: 2.1136 - val_categorical_accuracy: 0.2215
Epoch 2/6
609/609 [==============================] - 186s 305ms/step - loss: 1.6928 - categorical_accuracy: 0.3415 - val_loss: 2.0553 - val_categorical_accuracy: 0.2417
Epoch 3/6
609/609 [==============================] - 186s 305ms/step - loss: 1.6081 - categorical_accuracy: 0.3620 - val_loss: 1.8205 - val_categorical_accuracy: 0.3272
Epoch 4/6
609/609 [==============================] - 186s 306ms/step - loss: 1.4973 - categorical_accuracy: 0.3960 - val_loss: 1.8622 - val_categorical_accuracy: 0.2996
Epoch 5/6
609/609 [==============================] - 186s 305ms/step - loss: 1.4072 - categorical_accuracy: 0.4143 - val_loss: 1.8047 - val_categorical_accuracy: 0.3649
Epoch 6/6
609/609 [==============================] - 186s 305ms/step - loss: 1.3218 - categorical_accuracy: 0.4506 - val_loss: 1.8931 - val_categorical_accuracy: 0.3

#### Fold2: intentionally separate cells for check point purpose.

In [ ]:
from transformers import AutoTokenizer
import numpy as np

split0=splits[2]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
SEQ_LEN=256
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)
Xids_train.shape, Xids_test.shape:  (10832, 256) (2708, 256)


In [ ]:
drop_out_rate = 0.2
learning_rate_transfer_learning =1e-3
learning_rate_fine_tuning = 1e-5

In [ ]:
if len(balanced_accuracy_transfer_learning)==0:
  balanced_accuracy_transfer_learning = []
if len(balanced_accuracy_fine_tune)==0:
  balanced_accuracy_fine_tune = []

In [ ]:
McNemar.keys()

dict_keys([(False, 0.4, 0.0001, 1e-06), (True, 0.4, 0.0001, 1e-06), (False, 0.5, 0.001, 1e-05), (True, 0.5, 0.001, 1e-05), (False, 0.5, 0.001, 1e-06), (True, 0.5, 0.001, 1e-06), (False, 0.5, 0.0001, 1e-05), (True, 0.5, 0.0001, 1e-05), (False, 0.5, 0.0001, 1e-06), (True, 0.5, 0.0001, 1e-06), (False, 0.2, 0.001, 1e-05), (True, 0.2, 0.001, 1e-05)])

In [ ]:
from transformers import TFAutoModel

roberta_base = TFAutoModel.from_pretrained('roberta-base')
SEQ_LEN2=256
#step1
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = roberta_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model2.layers[2].trainable = False
print(model2.summary())

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
print(model2.summary() )#Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_transfer_learning.append(balanced_acc)


#step5: fine tune
model2.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_fine_tune.append(balanced_acc)
del(model2)


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_roberta_model_8 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 odel)                          thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

609/609 [==============================] - 205s 310ms/step - loss: 1.7873 - categorical_accuracy: 0.3000 - val_loss: 1.9561 - val_categorical_accuracy: 0.2638
Epoch 2/6
609/609 [==============================] - 186s 305ms/step - loss: 1.6619 - categorical_accuracy: 0.3348 - val_loss: 1.8618 - val_categorical_accuracy: 0.2950
Epoch 3/6
609/609 [==============================] - 186s 305ms/step - loss: 1.5407 - categorical_accuracy: 0.3714 - val_loss: 1.8462 - val_categorical_accuracy: 0.3088
Epoch 4/6
609/609 [==============================] - 186s 305ms/step - loss: 1.4429 - categorical_accuracy: 0.4050 - val_loss: 1.8335 - val_categorical_accuracy: 0.2960
Epoch 5/6
609/609 [==============================] - 186s 305ms/step - loss: 1.3527 - categorical_accuracy: 0.4263 - val_loss: 1.7527 - val_categorical_accuracy: 0.3465
Epoch 6/6
609/609 [==============================] - 186s 305ms/step - loss: 1.2430 - categorical_accuracy: 0.4650 - val_loss: 1.8267 - val_categorical_accuracy: 0.3

#### Fold3: intentionally separate cells for check point purpose.

In [ ]:
from transformers import AutoTokenizer
import numpy as np

split0=splits[3]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
SEQ_LEN=256
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)
Xids_train.shape, Xids_test.shape:  (10832, 256) (2708, 256)


In [ ]:
drop_out_rate = 0.2
learning_rate_transfer_learning =1e-3
learning_rate_fine_tuning = 1e-5

In [ ]:
if len(balanced_accuracy_transfer_learning)==0:
  balanced_accuracy_transfer_learning = []
if len(balanced_accuracy_fine_tune)==0:
  balanced_accuracy_fine_tune = []

In [ ]:
McNemar.keys()

dict_keys([(False, 0.4, 0.0001, 1e-06), (True, 0.4, 0.0001, 1e-06), (False, 0.5, 0.001, 1e-05), (True, 0.5, 0.001, 1e-05), (False, 0.5, 0.001, 1e-06), (True, 0.5, 0.001, 1e-06), (False, 0.5, 0.0001, 1e-05), (True, 0.5, 0.0001, 1e-05), (False, 0.5, 0.0001, 1e-06), (True, 0.5, 0.0001, 1e-06), (False, 0.2, 0.001, 1e-05), (True, 0.2, 0.001, 1e-05)])

In [ ]:
from transformers import TFAutoModel

roberta_base = TFAutoModel.from_pretrained('roberta-base')
SEQ_LEN2=256
#step1
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = roberta_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model2.layers[2].trainable = False
print(model2.summary())

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
print(model2.summary() )#Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_transfer_learning.append(balanced_acc)


#step5: fine tune
model2.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_fine_tune.append(balanced_acc)
del(model2)


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_roberta_model_9 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 odel)                          thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

609/609 [==============================] - 208s 314ms/step - loss: 1.7781 - categorical_accuracy: 0.3047 - val_loss: 2.2510 - val_categorical_accuracy: 0.1783
Epoch 2/6
609/609 [==============================] - 186s 306ms/step - loss: 1.6695 - categorical_accuracy: 0.3266 - val_loss: 2.0347 - val_categorical_accuracy: 0.2399
Epoch 3/6
609/609 [==============================] - 186s 306ms/step - loss: 1.5609 - categorical_accuracy: 0.3503 - val_loss: 2.0329 - val_categorical_accuracy: 0.2592
Epoch 4/6
609/609 [==============================] - 186s 305ms/step - loss: 1.4709 - categorical_accuracy: 0.3758 - val_loss: 1.8683 - val_categorical_accuracy: 0.3125
Epoch 5/6
609/609 [==============================] - 186s 305ms/step - loss: 1.3658 - categorical_accuracy: 0.4016 - val_loss: 1.9225 - val_categorical_accuracy: 0.3070
Epoch 6/6
609/609 [==============================] - 186s 305ms/step - loss: 1.2774 - categorical_accuracy: 0.4312 - val_loss: 1.9490 - val_categorical_accuracy: 0.2

#### Fold4: intentionally separate cells for check point purpose.

In [ ]:
from transformers import AutoTokenizer
import numpy as np

split0=splits[4]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
SEQ_LEN=256
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)
Xids_train.shape, Xids_test.shape:  (10832, 256) (2708, 256)


In [ ]:
drop_out_rate = 0.2
learning_rate_transfer_learning =1e-3
learning_rate_fine_tuning = 1e-5

In [ ]:
if len(balanced_accuracy_transfer_learning)==0:
  balanced_accuracy_transfer_learning = []
if len(balanced_accuracy_fine_tune)==0:
  balanced_accuracy_fine_tune = []

In [ ]:
McNemar.keys()

dict_keys([(False, 0.4, 0.0001, 1e-06), (True, 0.4, 0.0001, 1e-06), (False, 0.5, 0.001, 1e-05), (True, 0.5, 0.001, 1e-05), (False, 0.5, 0.001, 1e-06), (True, 0.5, 0.001, 1e-06), (False, 0.5, 0.0001, 1e-05), (True, 0.5, 0.0001, 1e-05), (False, 0.5, 0.0001, 1e-06), (True, 0.5, 0.0001, 1e-06), (False, 0.2, 0.001, 1e-05), (True, 0.2, 0.001, 1e-05)])

In [ ]:
from transformers import TFAutoModel

roberta_base = TFAutoModel.from_pretrained('roberta-base')
SEQ_LEN2=256
#step1
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = roberta_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model2.layers[2].trainable = False
print(model2.summary())

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
print(model2.summary() )#Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_transfer_learning.append(balanced_acc)


#step5: fine tune
model2.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
#y_pred_transfer_learning=get_balanced_accuracy(model2)
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracy_fine_tune.append(balanced_acc)
del(model2)


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_roberta_model_10 (TFRoberta  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 Model)                         thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                        

609/609 [==============================] - 207s 313ms/step - loss: 1.7845 - categorical_accuracy: 0.3351 - val_loss: 1.8349 - val_categorical_accuracy: 0.3355
Epoch 2/6
609/609 [==============================] - 186s 305ms/step - loss: 1.6569 - categorical_accuracy: 0.3653 - val_loss: 1.9392 - val_categorical_accuracy: 0.3162
Epoch 3/6
609/609 [==============================] - 186s 305ms/step - loss: 1.6362 - categorical_accuracy: 0.3610 - val_loss: 1.8225 - val_categorical_accuracy: 0.3529
Epoch 4/6
609/609 [==============================] - 186s 305ms/step - loss: 1.4867 - categorical_accuracy: 0.4071 - val_loss: 1.7600 - val_categorical_accuracy: 0.3759
Epoch 5/6
609/609 [==============================] - 186s 305ms/step - loss: 1.4101 - categorical_accuracy: 0.4310 - val_loss: 1.8395 - val_categorical_accuracy: 0.3079
Epoch 6/6
609/609 [==============================] - 186s 305ms/step - loss: 1.3372 - categorical_accuracy: 0.4553 - val_loss: 1.7881 - val_categorical_accuracy: 0.3

Take average

In [ ]:
balanced_accuracy_fine_tune

[0.3236895982367819,
 0.3586414884570302,
 0.3357361239116072,
 0.3775435303226758]

In [ ]:
balanced_accuracy_fine_tune.append(0.3560173405159383)

In [ ]:
sum(balanced_accuracy_fine_tune)/5

0.3503256162888067

### Don't need below

In [ ]:
from transformers import TFAutoModel
from transformers import AlbertTokenizer, TFAlbertModel
albert_base = TFAlbertModel.from_pretrained('albert-large-v2')

#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.3

#step1
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = albert_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model3 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model3.layers[2].trainable = False
#model2.summary()

#step2
optimizer = tf.keras.optimizers.Adam(1e-3)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model3.compile(optimizer=optimizer, loss= loss, metrics=metrics)
#model2.summary() #Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model3.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
balanced_acc=get_balanced_accuracy(model3)
balanced_accuracies_transfer_learning.append( balanced_acc )

#step5: fine tune
model3.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model3.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model3.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
balanced_acc=get_balanced_accuracy(model3)
balanced_accuracies_fine_tuning.append( balanced_acc )

del(model3)


Some layers from the model checkpoint at albert-large-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-large-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 427s 655ms/step - loss: 2.3814 - categorical_accuracy: 0.0950 - val_loss: 2.2728 - val_categorical_accuracy: 0.1857
Epoch 2/6
609/609 [==============================] - 393s 645ms/step - loss: 2.2444 - categorical_accuracy: 0.1341 - val_loss: 2.2647 - val_categorical_accuracy: 0.1278
Epoch 3/6
609/609 [==============================] - 393s 645ms/step - loss: 2.1293 - categorical_accuracy: 0.1591 - val_loss: 2.2699 - val_categorical_accuracy: 0.1324
Epoch 4/6
609/609 [==============================] - 393s 645ms/step - loss: 2.0163 - categorical_accuracy: 0.1886 - val_loss: 2.3037 - val_categorical_accuracy: 0.1195
              precision    recall  f1-score   support

         0.0       0.08      0.17      0.11       190
         1.0       0.08      0.17      0.11       173
         2.0       0.06      0.09      0.07       135
         3.0       0.10      0.06      0.07       216
         4.0       0.43      0.09      0.14      1036

609/609 [==============================] - 999s 2s/step - loss: 2.5387 - categorical_accuracy: 0.0855 - val_loss: 2.4253 - val_categorical_accuracy: 0.0708
Epoch 2/6
609/609 [==============================] - 960s 2s/step - loss: 2.3860 - categorical_accuracy: 0.0905 - val_loss: 2.2658 - val_categorical_accuracy: 0.1333
Epoch 3/6
609/609 [==============================] - 960s 2s/step - loss: 2.3354 - categorical_accuracy: 0.1109 - val_loss: 2.2675 - val_categorical_accuracy: 0.1949
Epoch 4/6
609/609 [==============================] - 960s 2s/step - loss: 2.4184 - categorical_accuracy: 0.0987 - val_loss: 2.3225 - val_categorical_accuracy: 0.2086
Epoch 5/6
609/609 [==============================] - 960s 2s/step - loss: 2.4178 - categorical_accuracy: 0.1224 - val_loss: 2.1792 - val_categorical_accuracy: 0.3575
Epoch 6/6
609/609 [==============================] - 960s 2s/step - loss: 2.3006 - categorical_accuracy: 0.2456 - val_loss: 2.2707 - val_categorical_accuracy: 0.0653
              

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from transformers import TFAutoModel
from transformers import AlbertTokenizer, TFAlbertModel
albert_base = TFAlbertModel.from_pretrained('albert-large-v2')

#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.4

#step1
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = albert_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model4 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model4.layers[2].trainable = False
#model2.summary()

#step2
optimizer = tf.keras.optimizers.Adam(1e-3)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model4.compile(optimizer=optimizer, loss= loss, metrics=metrics)
#model2.summary() #Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model4.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
balanced_acc=get_balanced_accuracy(model4)
balanced_accuracies_transfer_learning.append( balanced_acc )

#step5: fine tune
model4.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model4.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model4.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
balanced_acc=get_balanced_accuracy(model4)
balanced_accuracies_fine_tuning.append( balanced_acc )

del(model4)


Some layers from the model checkpoint at albert-large-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-large-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 426s 656ms/step - loss: 2.3735 - categorical_accuracy: 0.0898 - val_loss: 2.3128 - val_categorical_accuracy: 0.0616
Epoch 2/6
609/609 [==============================] - 393s 646ms/step - loss: 2.2869 - categorical_accuracy: 0.1220 - val_loss: 2.3036 - val_categorical_accuracy: 0.0818
Epoch 3/6
609/609 [==============================] - 393s 646ms/step - loss: 2.2384 - categorical_accuracy: 0.1312 - val_loss: 2.2920 - val_categorical_accuracy: 0.0910
Epoch 4/6
609/609 [==============================] - 393s 645ms/step - loss: 2.1644 - categorical_accuracy: 0.1486 - val_loss: 2.2738 - val_categorical_accuracy: 0.0965
Epoch 5/6
609/609 [==============================] - 394s 647ms/step - loss: 2.0775 - categorical_accuracy: 0.1607 - val_loss: 2.2763 - val_categorical_accuracy: 0.0928
Epoch 6/6
609/609 [==============================] - 394s 647ms/step - loss: 1.9924 - categorical_accuracy: 0.1802 - val_loss: 2.2644 - val_categorical_acc

### Conduct McNemar testing with the best parameter

In [ ]:
drop_out_rate = 0.2
learning_rate_transfer_learning = 1e-3
learning_rate_fine_tuning = 1e-5
McNemar = {}# (is_fine_tuning?, dropout_rate, learning_rate_transfer_learning, learning_rate_fine_tuning)

In [ ]:
from transformers import TFAutoModel

balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256

drop_out_rate = 0.2
learning_rate_transfer_learning = 1e-3
learning_rate_fine_tuning = 1e-5


print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
, learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
#step1
roberta = TFAutoModel.from_pretrained('roberta-base')
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = roberta(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model2.layers[2].trainable = False
#model2.summary()

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
#model2.summary() #Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_transfer_learning.append( balanced_acc )

#step5: fine tune
print("Fine tuning---------------")
model2.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_fine_tuning.append( balanced_acc )
print("----------------------------------------")
del(model2)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.2


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 158s 234ms/step - loss: 2.1384 - categorical_accuracy: 0.2122 - val_loss: 2.8267 - val_categorical_accuracy: 0.0901
Epoch 2/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9793 - categorical_accuracy: 0.2636 - val_loss: 2.3489 - val_categorical_accuracy: 0.1480
Epoch 3/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9232 - categorical_accuracy: 0.2825 - val_loss: 2.4198 - val_categorical_accuracy: 0.1379
Epoch 4/6
609/609 [==============================] - 138s 227ms/step - loss: 1.8901 - categorical_accuracy: 0.2974 - val_loss: 2.3096 - val_categorical_accuracy: 0.1581
Epoch 5/6
609/609 [==============================] - 138s 227ms/step - loss: 1.8674 - categorical_accuracy: 0.2979 - val_loss: 2.2364 - val_categorical_accuracy: 0.1801
Epoch 6/6
609/609 [==============================] - 138s 227ms/step - loss: 1.8519 - categorical_accuracy: 0.3018 - val_loss: 2.2021 - val_categorical_acc

609/609 [==============================] - 357s 553ms/step - loss: 1.7825 - categorical_accuracy: 0.3348 - val_loss: 1.8293 - val_categorical_accuracy: 0.2960
Epoch 2/6
609/609 [==============================] - 333s 547ms/step - loss: 1.6608 - categorical_accuracy: 0.3509 - val_loss: 1.8948 - val_categorical_accuracy: 0.2923
Epoch 3/6
609/609 [==============================] - 333s 547ms/step - loss: 1.5689 - categorical_accuracy: 0.3829 - val_loss: 2.0517 - val_categorical_accuracy: 0.2491
Epoch 4/6
609/609 [==============================] - 333s 547ms/step - loss: 1.4769 - categorical_accuracy: 0.4039 - val_loss: 1.9109 - val_categorical_accuracy: 0.2978
Epoch 5/6
609/609 [==============================] - 333s 547ms/step - loss: 1.3818 - categorical_accuracy: 0.4348 - val_loss: 1.7944 - val_categorical_accuracy: 0.3529
Epoch 6/6
609/609 [==============================] - 333s 547ms/step - loss: 1.2942 - categorical_accuracy: 0.4637 - val_loss: 1.7972 - val_categorical_accuracy: 0.3

In [ ]:
McNemar_RoBERTa_split0_transfer_learning = pd.DataFrame(data= McNemar[(False,0.2, 1e-3,1e-5)] )
McNemar_RoBERTa_split0_transfer_learning.to_csv(DIR + 'McNemar_RoBERTa_split0_transfer_learning.csv')

McNemar_RoBERTa_split0_fine_tuning = pd.DataFrame(data= McNemar[(True, 0.2, 1e-3,1e-5)] )
McNemar_RoBERTa_split0_fine_tuning.to_csv(DIR + 'McNemar_RoBERTa_split0_fine_tuning.csv')

### Unnecessary code

In [ ]:
split0['X_test'] = prepare_lyrics(split0['X_test'] )
split0['X_test']

2        A buh A buh You went to school to learn girl T...
4        Say the words I cannot say Say them on another...
5        I was alone I was made of stone You took me ho...
9        Again the burden of losing rests upon my shoul...
20       only been three weeks And a bag of speed from ...
                               ...                        
13517    Like the legend of the Phoenix All ends with b...
13522    Mr Telephone man something wrong with my line ...
13526    can you imagine what it would be like we never...
13532    Love of my life hurt me broken my heart and no...
13535    think what afraid of come in you know been mad...
Name: texts, Length: 2708, dtype: object

In [ ]:
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_train'] 

0        a sunny day so I got nowhere to hide Not a clo...
1        Tell me a tale that always was Sing me a song ...
3        like a conversation where stops to breathe Is ...
6        Locked up tight Like I would never feel again ...
7        sittin in the crib dreamin about leer jets and...
                               ...                        
13534    grandma cookies nigga Shout out to fronto leaf...
13536    Oh yeah yeah Last night I took a walk in the s...
13537    Innocence it come easy in a sense it never wil...
13538    Girl you know how I feel I really Since you be...
13539    wwI oh must go on standing You break that whic...
Name: texts, Length: 10832, dtype: object

In [ ]:
split0['y_train'] 

0        4
1        9
3        4
6        4
7        6
        ..
13534    6
13536    4
13537    3
13538    4
13539    8
Name: labels, Length: 10832, dtype: int8

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

import numpy as np
Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']
print(Xids_train.shape, Xids_test.shape)

(10832, 256) (2708, 256)


In [ ]:
Xids_train[0]

array([  101.,   170., 21162.,  1285.,  1177.,   146.,  1400.,  8251.,
        1106.,  4750.,  1753.,   170.,  7180.,  1107.,  1103.,  3901.,
        1573., 12714.,  1400.,  1380.,  1107.,  1139.,  1257.,  2066.,
        1177.,  1128.,  1267.,  1143.,  5354.,  1573.,   146.,  3074.,
        1111.,  1103.,  4162.,   184., 10559.,  1706.,  4750.,  1155.,
        1139.,  3632.,  1133.,  1781.,  1315.,  1263.,  1252.,   146.,
        1576.,  1111.,  1103.,  1442.,   184., 10559.,  1191.,  1128.,
        1341.,  4780.,  1173.,  1115.,  2228.,  1128.,  2012.,  4458.,
        1113.,  1103.,  3945.,  1104.,  1113.,  1176.,  2613.,  1107.,
         170.,  2937.,  4842.,  4458.,  1113.,  1129.,  2288.,  1223.,
        4458.,  1113., 12120.,  1116., 13830.,  4862.,  1139.,  3632.,
        1177.,  1115.,  1309.,  1267.,  2421.,  1122.,  4458.,  1122.,
        4458.,  1519.,  1122.,  4458.,  1113.,  1143.,  1519.,  1122.,
        4458.,  1519.,  1122.,  4458.,  1113.,  2421.,  1122.,  4458.,
      

Create y labels in transformer format

In [ ]:
np.set_printoptions(threshold=np.inf)

In [ ]:
labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))
"""
for Xids_train, Xmask_train, labels_train in dataset_train.take(1):
  print(labels_train)

print(labels_train.shape)
"""

'\nfor Xids_train, Xmask_train, labels_train in dataset_train.take(1):\n  print(labels_train)\n\nprint(labels_train.shape)\n'

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

import numpy as np

Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

In [ ]:
len(dataset_train)

10832

In [ ]:
dataset_train = dataset_train.map(map_func)

for i in dataset_train.take(1):
  print(i)

({'input_ids': <tf.Tensor: shape=(256,), dtype=float64, numpy=
array([  101.,   170., 21162.,  1285.,  1177.,   146.,  1400.,  8251.,
        1106.,  4750.,  1753.,   170.,  7180.,  1107.,  1103.,  3901.,
        1573., 12714.,  1400.,  1380.,  1107.,  1139.,  1257.,  2066.,
        1177.,  1128.,  1267.,  1143.,  5354.,  1573.,   146.,  3074.,
        1111.,  1103.,  4162.,   184., 10559.,  1706.,  4750.,  1155.,
        1139.,  3632.,  1133.,  1781.,  1315.,  1263.,  1252.,   146.,
        1576.,  1111.,  1103.,  1442.,   184., 10559.,  1191.,  1128.,
        1341.,  4780.,  1173.,  1115.,  2228.,  1128.,  2012.,  4458.,
        1113.,  1103.,  3945.,  1104.,  1113.,  1176.,  2613.,  1107.,
         170.,  2937.,  4842.,  4458.,  1113.,  1129.,  2288.,  1223.,
        4458.,  1113., 12120.,  1116., 13830.,  4862.,  1139.,  3632.,
        1177.,  1115.,  1309.,  1267.,  2421.,  1122.,  4458.,  1122.,
        4458.,  1519.,  1122.,  4458.,  1113.,  1143.,  1519.,  1122.,
        4458.,

In [ ]:
dataset_test = dataset_test.map(map_func)
dataset_test

<MapDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(10,), dtype=tf.float64, name=None))>

In [ ]:
dataset_train = dataset_train.shuffle(42).batch(32)
DS_LEN = len(list(dataset_train))
DS_LEN

339

339 because 10800/32

In [ ]:
SPLIT = 0.9
round(DS_LEN*SPLIT)

305

In [ ]:
len(split0['X_train'])-int(len(split0['X_train'])*0.9)

1084

In [ ]:
train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))#remainer is for val.

In [ ]:
test = dataset_test.batch(32)

In [ ]:
def map_func_only_X(val_dictionary, labels):
  return {'input_ids': val_dictionary['input_ids'], 'attention_mask':val_dictionary['attention_mask']}

In [ ]:
def map_func_only_y(val_dictionary, labels):
  return labels

In [ ]:
VAL_SIZE=1088

### Imbalance

In [ ]:
MAX = max(counter.values())
MAX

4143

In [ ]:
my_weight={}

for genre in range(10):
  my_weight[genre]=MAX/counter[genre]
#sum(weight)
my_weight

{0: 5.4298820445609435,
 1: 6.004347826086956,
 2: 7.715083798882682,
 3: 4.789595375722543,
 4: 1.0,
 5: 13.53921568627451,
 6: 5.291187739463601,
 7: 3.5746333045729077,
 8: 7.451438848920863,
 9: 4.022330097087378}

In [ ]:
from transformers import TFAutoModel
bert = TFAutoModel.from_pretrained('bert-base-cased')

input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask')

embeddings = bert(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model.layers[2].trainable = False
model.summary()

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

BERT is huge, so here we don't fine tune BERT. Simply use pre-trained base BERT

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-3)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model.compile(optimizer=optimizer, loss= loss, metrics=metrics)

In [ ]:
print(DS_LEN, len(train), len(val), len(test))

339 305 34 85


In [ ]:
VAL_SIZE = len(val)*32
VAL_SIZE

1088

In [ ]:
#print(len(val.take(len(val))))
for item, label_batch in val.take(1):#len(val)
  print(item['input_ids'])
  #one_batch= item[0]['input_ids']
  #for xids in one_batch:
  #  print(xids.shape)

In [ ]:
val_X = val.map(map_func_only_X)
print(len(val_X))
#for i in val_X.take(1):
  #print(i)

34


In [ ]:
val_y = val.map(map_func_only_y)
print(len(val_y))
y_category=np.zeros((VAL_SIZE, ))
counter=0
for label_tensor in val_y.take(1):
  #print(label_tensor)
  y_val = np.argmax(label_tensor, axis=1)
  for label in y_val:
    print(label) 
    y_category[counter]=label
    counter+=1
print('counter:',counter)
print(y_category)

In [ ]:
VAL_SIZE=1072
VAL_SIZE

1072

In [ ]:
VAL_SIZE=len(split0['X_train'])-int(len(split0['X_train'])*0.9)
VAL_SIZE

1084

In [ ]:
val_y = val2.map(map_func_only_y)
print(len(val_y))
y_category=np.zeros((VAL_SIZE, ))
counter=0
for label_tensor in val_y.take(len(val2)):
  #print(label_tensor)
  y_val = np.argmax(label_tensor, axis=1)
  for label in y_val:
    print(label) 
    y_category[counter]=label
    counter+=1
print('counter:',counter)
print(y_category)

68
4
6
2
5
4
6
7
1
0
7
3
4
4
7
4
4
9
0
4
5
8
7
5
4
4
3
3
0
4
7
9
1
2
7
9
4
4
4
4
6
6
4
4
8
4
0
3
4
0
9
4
4
9
4
0
4
4
9
6
7
2
4
4
9
4
6
6
3
7
4
4
2
0
3
1
0
3
4
9
4
3
9
8
4
8
5
4
4
4
3
4
4
3
1
4
7
4
9
4
5
8
4
3
9
4
4
4
4
2
7
7
0
7
0
4
4
3
1
7
4
4
7
9
4
4
4
7
9
4
8
4
7
7
5
1
4
4
4
9
4
1
2
4
6
7
4
4
0
7
6
0
4
5
7
4
4
9
9
2
4
9
6
4
0
7
1
4
4
0
4
4
0
7
4
7
7
4
3
3
8
4
3
6
1
6
7
2
2
2
0
6
7
4
4
7
4
2
4
7
3
4
5
7
5
4
3
4
6
7
2
7
6
4
7
6
7
9
1
7
4
4
6
6
6
3
4
0
5
1
0
6
7
3
5
4
5
4
1
6
4
1
8
4
8
6
7
9
9
2
9
4
4
7
4
4
7
4
4
4
4
3
4
0
8
4
4
9
4
7
9
4
4
4
0
6
4
4
6
7
0
9
7
6
4
4
4
9
6
4
7
9
8
7
4
3
4
4
0
9
4
4
4
4
3
7
4
0
7
4
4
2
0
3
4
8
0
4
2
0
8
3
0
6
2
4
1
4
5
4
4
4
8
3
6
4
7
4
4
1
4
4
0
9
9
6
3
4
3
4
4
4
4
1
4
8
2
3
6
4
3
4
7
7
9
7
3
4
4
9
1
1
4
4
9
6
3
4
6
3
2
4
5
4
4
9
4
3
2
8
4
4
4
4
7
7
8
4
7
9
0
6
8
4
1
9
0
7
4
2
8
9
9
4
4
4
6
8
9
7
4
9
4
7
4
0
4
4
9
8
4
1
7
3
1
7
7
0
0
0
2
4
4
8
4
4
7
4
3
4
2
4
2
8
9
4
1
8
9
4
4
4
9
3
4
2
8
7
4
7
1
0
9
6
4
1
9
4
9
5
7
2
4
4
7
6
9
7
4
9
3
9
3
9
7
4
9
7
4
1

Ref1: how to calculate genre as int instead of one hot vector from list of one hot vectors.
https://www.sharpsightlabs.com/blog/numpy-argmax/

Ref2: How to use self-programmed metrics: https://stackoverflow.com/questions/37657260/how-to-implement-custom-metric-in-keras



In [ ]:
VAL_SIZE=1072

In [ ]:
from sklearn.metrics import balanced_accuracy_score
class Metrics(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self._data = []
        self.validation_data = val2

    def map_func_only_X(val_dictionary, labels):
      return {'input_ids': val_dictionary['input_ids'], 'attention_mask':val_dictionary['attention_mask']}
    def map_func_only_y(val_dictionary, labels):
      return labels

    def on_epoch_end(self, batch, logs={}):
      #Xid_list= np.zeros((VAL_SIZE, SEQ_LEN))
      #y_onehot_list= np.zeros((VAL_SIZE, 10))
      #counter=0
      #for item in dataset.take(len(self.validation_data)):
      #for item in val.take(len(val)):
      """
      for item, batch_label in self.validation_data.take(len(val)):
        for label in batch_label:
          y_onehot_list[counter,:] = label
          counter+=1
        one_batch_xid = item['input_ids']
        for xids in one_batch_xid:
          Xid_list[counter, :] = xids
        
        #X_val, y_val = self.validation_data[0], self.validation_data[1]
      """
      val_y = self.validation_data.map(map_func_only_y)
      y_category=np.zeros((VAL_SIZE, ))
      counter=0
      for label_tensor in val_y.take(len(val_y)):
        y_val = np.argmax(label_tensor, axis=1)
        for label in y_val:
          y_category[counter]=label
          counter+=1
      X_val, y_val =self.validation_data.map(map_func_only_X), y_category
      y_predict = np.asarray(model2.predict(X_val))

      #y_val = np.argmax(y_val, axis=1)#I have already done converting to category
      y_predict = np.argmax(y_predict, axis=1)

      self._data.append({
          'balanced_accuracy': balanced_accuracy_score(y_val, y_predict),
          #'val_rocauc': roc_auc_score(y_val, y_predict),
      })
      return

    def get_data(self):
        return self._data

### Transfer learning: try early stopping with SEQ_LEN=512

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)

metrics = Metrics()
history = model.fit(train, validation_data=val, epochs=10, class_weight=my_weight ,callbacks=[metrics, early_stopping])
metrics.get_data()

Epoch 1/10
305/305 [==============================] - 165s 541ms/step - loss: 7.4384 - categorical_accuracy: 0.2784 - val_loss: 2.1002 - val_categorical_accuracy: 0.2118
Epoch 2/10
305/305 [==============================] - 165s 541ms/step - loss: 7.0577 - categorical_accuracy: 0.3053 - val_loss: 2.0583 - val_categorical_accuracy: 0.2435
Epoch 3/10
305/305 [==============================] - 165s 542ms/step - loss: 6.7958 - categorical_accuracy: 0.3207 - val_loss: 1.9879 - val_categorical_accuracy: 0.2677
Epoch 4/10
305/305 [==============================] - 165s 542ms/step - loss: 6.5908 - categorical_accuracy: 0.3376 - val_loss: 2.0138 - val_categorical_accuracy: 0.2388
Epoch 5/10
305/305 [==============================] - 165s 542ms/step - loss: 6.4173 - categorical_accuracy: 0.3493 - val_loss: 2.0363 - val_categorical_accuracy: 0.2472
Epoch 6/10
305/305 [==============================] - 165s 542ms/step - loss: 6.2501 - categorical_accuracy: 0.3629 - val_loss: 2.0147 - val_categoric

[{'balanced_accuracy': 0.10064757124688836},
 {'balanced_accuracy': 0.1187677795399048},
 {'balanced_accuracy': 0.09990412917019217},
 {'balanced_accuracy': 0.08602748843296298},
 {'balanced_accuracy': 0.12293896335258817},
 {'balanced_accuracy': 0.08466265273217208}]

In [ ]:
TEST_SIZE = split0["y_test"].shape[0]
TEST_SIZE

2708

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score
test_y = test.map(map_func_only_y)
y_category=np.zeros((TEST_SIZE, ))
counter=0
for label_tensor in test_y.take(len(test_y)):
  y_test = np.argmax(label_tensor, axis=1)
  for label in y_test:
    y_category[counter]=label
    counter+=1

X_test, y_test = test.map(map_func_only_X), y_category
y_predict_test = np.asarray(model.predict(X_test))
y_predict_test = np.argmax(y_predict_test, axis=1)
print(classification_report(y_test, y_predict_test) )
print(balanced_accuracy_score(y_test, y_predict_test))

              precision    recall  f1-score   support

         0.0       0.14      0.42      0.21       190
         1.0       0.16      0.28      0.20       173
         2.0       0.10      0.14      0.12       135
         3.0       0.13      0.17      0.15       216
         4.0       0.74      0.19      0.31      1036
         5.0       0.12      0.38      0.18        76
         6.0       0.77      0.77      0.77       195
         7.0       0.11      0.08      0.10       290
         8.0       0.09      0.09      0.09       139
         9.0       0.24      0.28      0.26       258

    accuracy                           0.25      2708
   macro avg       0.26      0.28      0.24      2708
weighted avg       0.42      0.25      0.26      2708

0.2825509565558356


### Transfer learning -> Fine tunine: unfreeze middle layers and train with smaller learning rate.

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
SEQ_LEN2=256
import numpy as np
Xids_train2 = np.zeros((split0['X_train'].shape[0], SEQ_LEN2))
Xmask_train2 = np.zeros((split0['X_train'].shape[0], SEQ_LEN2))
#Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
#Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN2, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train2[i,:], Xmask_train2[i,:] = tokens['input_ids'], tokens['attention_mask']

#for i, lyric in enumerate(split0['X_test']):
#  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN2, truncation =True, padding="max_length"
#    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
#  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']
print(Xids_train2.shape,)

(10832, 256)


In [ ]:
from transformers import TFAutoModel
roberta = TFAutoModel.from_pretrained('roberta-base')

input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = roberta(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model2.layers[2].trainable = False
model2.summary()

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 el)                            thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
print(Xids_train2.shape)

(10832, 256)


In [ ]:
labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_train

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 

In [ ]:
Xids_train2[0]

In [ ]:
#labels_train = np.zeros((split0['y_train'].shape[0], 10))
#labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1

dataset_train2 = tf.data.Dataset.from_tensor_slices((Xids_train2, Xmask_train2, labels_train))
dataset_train2 = dataset_train2.map(map_func)
dataset_train2 = dataset_train2.shuffle(42).batch(16)

SPLIT=0.9
DS_LEN = len(dataset_train2)
print("DS_LEN:", DS_LEN)

train2 = dataset_train2.take(round(DS_LEN*SPLIT))
val2 = dataset_train2.skip(round(DS_LEN*SPLIT))#remainer is for val.

DS_LEN: 677


In [ ]:
print(len(train2), len(val2))

609 68


First, transfer learning

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-3)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
model2.summary() #Check trainable params increased.

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 el)                            thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
VAL_SIZE=1088

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)

#metrics = Metrics()
history = model2.fit(train2, validation_data=val2, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])#metrics
#metrics.get_data()

Epoch 1/6
609/609 [==============================] - 94s 134ms/step - loss: 2.1050 - categorical_accuracy: 0.2384 - val_loss: 2.5287 - val_categorical_accuracy: 0.1305
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9531 - categorical_accuracy: 0.2923 - val_loss: 2.6360 - val_categorical_accuracy: 0.1167
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9067 - categorical_accuracy: 0.2974 - val_loss: 2.3261 - val_categorical_accuracy: 0.1434
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 1.8769 - categorical_accuracy: 0.3051 - val_loss: 2.2580 - val_categorical_accuracy: 0.1682
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 1.8385 - categorical_accuracy: 0.3131 - val_loss: 2.3655 - val_categorical_accuracy: 0.1811
Epoch 6/6
609/609 [==============================] - 79s 129ms/step - loss: 1.8289 - categorical_accuracy: 0.3206 - val_loss: 2.1423 - val_categorical_accuracy:

In [ ]:
TEST_SIZE = split0["y_test"].shape[0]
TEST_SIZE

2708

In [ ]:
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN2))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN2))
for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN2, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))
dataset_test = dataset_test.map(map_func)
test = dataset_test.batch(16)

In [ ]:
get_balanced_accuracy(model2)

              precision    recall  f1-score   support

         0.0       0.14      0.15      0.15       190
         1.0       0.18      0.14      0.16       173
         2.0       0.07      0.05      0.06       135
         3.0       0.14      0.25      0.18       216
         4.0       0.75      0.03      0.07      1036
         5.0       0.08      0.58      0.15        76
         6.0       0.43      0.84      0.57       195
         7.0       0.19      0.03      0.06       290
         8.0       0.09      0.29      0.14       139
         9.0       0.22      0.38      0.28       258

    accuracy                           0.19      2708
   macro avg       0.23      0.28      0.18      2708
weighted avg       0.40      0.19      0.15      2708

0.27578776890249534


Fine tuning with smaller learning rate

In [ ]:
# Unfreeze the base model
#model.trainable = True
model2.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
model2.summary() #Check trainable params increased.

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 el)                            thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
#metrics = Metrics()
history = model2.fit(train2, validation_data=val2, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])#metrics
#metrics.get_data()

Epoch 1/6


609/609 [==============================] - 203s 307ms/step - loss: 1.7488 - categorical_accuracy: 0.3503 - val_loss: 1.9209 - val_categorical_accuracy: 0.3309
Epoch 2/6
609/609 [==============================] - 185s 303ms/step - loss: 1.6219 - categorical_accuracy: 0.3758 - val_loss: 1.8664 - val_categorical_accuracy: 0.3373
Epoch 3/6
609/609 [==============================] - 185s 303ms/step - loss: 1.5014 - categorical_accuracy: 0.4157 - val_loss: 1.8434 - val_categorical_accuracy: 0.2960
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 1.4113 - categorical_accuracy: 0.4380 - val_loss: 1.8316 - val_categorical_accuracy: 0.3373
Epoch 5/6
609/609 [==============================] - 185s 303ms/step - loss: 1.2980 - categorical_accuracy: 0.4698 - val_loss: 1.8991 - val_categorical_accuracy: 0.3419
Epoch 6/6
609/609 [==============================] - 185s 304ms/step - loss: 1.2123 - categorical_accuracy: 0.5004 - val_loss: 1.7828 - val_categorical_accuracy: 0.3

In [ ]:
get_balanced_accuracy(model2)

              precision    recall  f1-score   support

         0.0       0.26      0.23      0.24       190
         1.0       0.19      0.32      0.24       173
         2.0       0.09      0.25      0.13       135
         3.0       0.25      0.20      0.22       216
         4.0       0.74      0.41      0.53      1036
         5.0       0.23      0.41      0.30        76
         6.0       0.65      0.90      0.76       195
         7.0       0.14      0.01      0.01       290
         8.0       0.13      0.23      0.17       139
         9.0       0.27      0.47      0.34       258

    accuracy                           0.36      2708
   macro avg       0.30      0.34      0.29      2708
weighted avg       0.44      0.36      0.36      2708

0.34310539549600694


### Try different learning rate and class weight

In [ ]:
#Tutorial
#weight_for_0 = (1 / neg) * (total / 2.0)
#weight_for_1 = (1 / pos) * (total / 2.0)

counter = Counter(split0['y_train'])
my_weight2 = {}
print(counter)
print("SUM:", SUM)
for genre in counter:
  #print(genre, counter[genre])
  my_weight2[genre] = (1/counter[genre]) * (SUM/10)
my_weight2

Counter({4: 4143, 7: 1159, 9: 1030, 3: 865, 6: 783, 0: 763, 1: 690, 8: 556, 2: 537, 5: 306})
SUM: 10832


{4: 0.2614530533429882,
 9: 1.051650485436893,
 6: 1.383397190293742,
 2: 2.0171322160148977,
 0: 1.4196592398427261,
 7: 0.9345987920621226,
 1: 1.569855072463768,
 5: 3.539869281045752,
 8: 1.948201438848921,
 3: 1.2522543352601156}

In [ ]:
#learning_rate=1e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)#New!
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model.compile(optimizer=optimizer, loss= loss, metrics=metrics)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)

metrics = Metrics()
history = model.fit(train, validation_data=val, epochs=10, class_weight=my_weight2 ,callbacks=[metrics, early_stopping])
metrics.get_data()

Epoch 1/10
305/305 [==============================] - 184s 561ms/step - loss: 2.6286 - categorical_accuracy: 0.1101 - val_loss: 2.4429 - val_categorical_accuracy: 0.1101
Epoch 2/10
305/305 [==============================] - 166s 543ms/step - loss: 2.4978 - categorical_accuracy: 0.1270 - val_loss: 2.5052 - val_categorical_accuracy: 0.1175
Epoch 3/10
305/305 [==============================] - 165s 543ms/step - loss: 2.4273 - categorical_accuracy: 0.1461 - val_loss: 2.4666 - val_categorical_accuracy: 0.1362
Epoch 4/10
305/305 [==============================] - 166s 543ms/step - loss: 2.3810 - categorical_accuracy: 0.1579 - val_loss: 2.4322 - val_categorical_accuracy: 0.1455
Epoch 5/10
305/305 [==============================] - 165s 543ms/step - loss: 2.3313 - categorical_accuracy: 0.1748 - val_loss: 2.4089 - val_categorical_accuracy: 0.1595
Epoch 6/10
305/305 [==============================] - 165s 543ms/step - loss: 2.3018 - categorical_accuracy: 0.1837 - val_loss: 2.3841 - val_categoric

[{'balanced_accuracy': 0.09345675855115052},
 {'balanced_accuracy': 0.10264778383673703},
 {'balanced_accuracy': 0.09508081992184837},
 {'balanced_accuracy': 0.12308197160332786},
 {'balanced_accuracy': 0.09630914802065789},
 {'balanced_accuracy': 0.10347840566037467},
 {'balanced_accuracy': 0.11210863180805575},
 {'balanced_accuracy': 0.0786235643867815},
 {'balanced_accuracy': 0.11978332480356244},
 {'balanced_accuracy': 0.10973488557563418}]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score

TEST_SIZE = split0["y_test"].shape[0]
test_y = test.map(map_func_only_y)
y_category=np.zeros((TEST_SIZE, ))
counter=0
for label_tensor in test_y.take(len(test_y)):
  y_test = np.argmax(label_tensor, axis=1)
  for label in y_test:
    y_category[counter]=label
    counter+=1

X_test, y_test = test.map(map_func_only_X), y_category
y_predict_test = np.asarray(model.predict(X_test))
y_predict_test = np.argmax(y_predict_test, axis=1)

print(classification_report(y_test, y_predict_test) )
print(balanced_accuracy_score(y_test, y_predict_test))

              precision    recall  f1-score   support

         0.0       0.09      0.14      0.11       190
         1.0       0.13      0.17      0.15       173
         2.0       0.06      0.16      0.09       135
         3.0       0.08      0.11      0.09       216
         4.0       0.65      0.16      0.25      1036
         5.0       0.04      0.22      0.07        76
         6.0       0.41      0.64      0.50       195
         7.0       0.11      0.07      0.09       290
         8.0       0.08      0.18      0.11       139
         9.0       0.12      0.03      0.05       258

    accuracy                           0.17      2708
   macro avg       0.18      0.19      0.15      2708
weighted avg       0.33      0.17      0.18      2708

0.1872912453963657


### Try Learning rate=0.001

In [ ]:
#learning_rate=1e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)#New!
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model.compile(optimizer=optimizer, loss= loss, metrics=metrics)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)

metrics = Metrics()
history = model.fit(train, validation_data=val, epochs=10, class_weight=my_weight2 ,callbacks=[metrics, early_stopping])
metrics.get_data()

Epoch 1/10
305/305 [==============================] - 184s 561ms/step - loss: 2.0906 - categorical_accuracy: 0.2533 - val_loss: 2.1767 - val_categorical_accuracy: 0.1950
Epoch 2/10
305/305 [==============================] - 165s 542ms/step - loss: 1.9176 - categorical_accuracy: 0.2940 - val_loss: 2.1084 - val_categorical_accuracy: 0.2267
Epoch 3/10
305/305 [==============================] - 165s 542ms/step - loss: 1.8304 - categorical_accuracy: 0.3076 - val_loss: 2.0874 - val_categorical_accuracy: 0.2631
Epoch 4/10
305/305 [==============================] - 165s 542ms/step - loss: 1.7569 - categorical_accuracy: 0.3246 - val_loss: 2.0556 - val_categorical_accuracy: 0.2528
Epoch 5/10
305/305 [==============================] - 165s 543ms/step - loss: 1.6983 - categorical_accuracy: 0.3422 - val_loss: 2.0449 - val_categorical_accuracy: 0.2677
Epoch 6/10
305/305 [==============================] - 165s 542ms/step - loss: 1.6661 - categorical_accuracy: 0.3546 - val_loss: 2.0368 - val_categoric

[{'balanced_accuracy': 0.10015323622516308},
 {'balanced_accuracy': 0.09505681176810313},
 {'balanced_accuracy': 0.0997062311977546},
 {'balanced_accuracy': 0.10316824160557166},
 {'balanced_accuracy': 0.09387677165808059},
 {'balanced_accuracy': 0.10081253716428092},
 {'balanced_accuracy': 0.11033378948094193},
 {'balanced_accuracy': 0.1003793423366475}]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score

TEST_SIZE = split0["y_test"].shape[0]
test_y = test.map(map_func_only_y)
y_category=np.zeros((TEST_SIZE, ))
counter=0
for label_tensor in test_y.take(len(test_y)):
  y_test = np.argmax(label_tensor, axis=1)
  for label in y_test:
    y_category[counter]=label
    counter+=1

X_test, y_test = test.map(map_func_only_X), y_category
y_predict_test = np.asarray(model.predict(X_test))
y_predict_test = np.argmax(y_predict_test, axis=1)

print(classification_report(y_test, y_predict_test) )
print(balanced_accuracy_score(y_test, y_predict_test))

              precision    recall  f1-score   support

         0.0       0.11      0.19      0.14       190
         1.0       0.15      0.20      0.17       173
         2.0       0.11      0.30      0.16       135
         3.0       0.12      0.10      0.11       216
         4.0       0.77      0.22      0.34      1036
         5.0       0.16      0.43      0.23        76
         6.0       0.78      0.74      0.76       195
         7.0       0.08      0.04      0.05       290
         8.0       0.09      0.29      0.13       139
         9.0       0.26      0.30      0.28       258

    accuracy                           0.25      2708
   macro avg       0.26      0.28      0.24      2708
weighted avg       0.42      0.25      0.27      2708

0.2813560671358631


### Try dropout rate 0.2

In [ ]:
from transformers import TFAutoModel
bert = TFAutoModel.from_pretrained('bert-base-cased')

input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask')

embeddings = bert(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model.layers[2].trainable = False
model.summary()

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

BERT is huge, so here we don't fine tune BERT. Simply use pre-trained base BERT

In [ ]:
#learning_rate=1e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)#New!
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model.compile(optimizer=optimizer, loss= loss, metrics=metrics)

In [ ]:
#Tutorial
#weight_for_0 = (1 / neg) * (total / 2.0)
#weight_for_1 = (1 / pos) * (total / 2.0)

counter = Counter(split0['y_train'])
my_weight2 = {}
print(counter)
print("SUM:", SUM)
for genre in counter:
  #print(genre, counter[genre])
  my_weight2[genre] = (1/counter[genre]) * (SUM/10)
my_weight2

Counter({4: 4143, 7: 1159, 9: 1030, 3: 865, 6: 783, 0: 763, 1: 690, 8: 556, 2: 537, 5: 306})
SUM: 10832


{4: 0.2614530533429882,
 9: 1.051650485436893,
 6: 1.383397190293742,
 2: 2.0171322160148977,
 0: 1.4196592398427261,
 7: 0.9345987920621226,
 1: 1.569855072463768,
 5: 3.539869281045752,
 8: 1.948201438848921,
 3: 1.2522543352601156}

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)

metrics = Metrics()
history = model.fit(train, validation_data=val, epochs=10, class_weight=my_weight2 ,callbacks=[metrics, early_stopping])
metrics.get_data()

Epoch 1/10
305/305 [==============================] - 184s 560ms/step - loss: 2.1479 - categorical_accuracy: 0.2176 - val_loss: 2.0500 - val_categorical_accuracy: 0.2528
Epoch 2/10
305/305 [==============================] - 165s 543ms/step - loss: 1.9392 - categorical_accuracy: 0.2829 - val_loss: 2.0951 - val_categorical_accuracy: 0.2071
Epoch 3/10
305/305 [==============================] - 165s 542ms/step - loss: 1.8694 - categorical_accuracy: 0.2932 - val_loss: 2.1129 - val_categorical_accuracy: 0.2239
Epoch 4/10
305/305 [==============================] - 165s 542ms/step - loss: 1.8010 - categorical_accuracy: 0.3207 - val_loss: 2.1080 - val_categorical_accuracy: 0.2332


[{'balanced_accuracy': 0.09008432702203581},
 {'balanced_accuracy': 0.10269493309177288},
 {'balanced_accuracy': 0.10311956617388633},
 {'balanced_accuracy': 0.11352685588509814}]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score

TEST_SIZE = split0["y_test"].shape[0]
test_y = test.map(map_func_only_y)
y_category=np.zeros((TEST_SIZE, ))
counter=0
for label_tensor in test_y.take(len(test_y)):
  y_test = np.argmax(label_tensor, axis=1)
  for label in y_test:
    y_category[counter]=label
    counter+=1

X_test, y_test = test.map(map_func_only_X), y_category
y_predict_test = np.asarray(model.predict(X_test))
y_predict_test = np.argmax(y_predict_test, axis=1)

print(classification_report(y_test, y_predict_test) )
print(balanced_accuracy_score(y_test, y_predict_test))

              precision    recall  f1-score   support

         0.0       0.12      0.11      0.12       190
         1.0       0.14      0.23      0.17       173
         2.0       0.08      0.23      0.12       135
         3.0       0.14      0.06      0.08       216
         4.0       0.74      0.21      0.32      1036
         5.0       0.13      0.33      0.19        76
         6.0       0.79      0.75      0.77       195
         7.0       0.11      0.06      0.08       290
         8.0       0.11      0.47      0.18       139
         9.0       0.20      0.26      0.23       258

    accuracy                           0.24      2708
   macro avg       0.26      0.27      0.23      2708
weighted avg       0.41      0.24      0.26      2708

0.2709493430228308


Dropout 0.2 was actually worse than Dropout=0.1